# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average* per year of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [298]:
# Import panda
import os, sys
import pandas as pd
import numpy as np
import csv
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.
DATA_DIRECTORY = './Data/ebola/'

In [299]:
def get_csv_in_chrono_order(folder_path):
    filenames = pd.Series(os.listdir(folder_path)).sort_values()
    filenames.index = range(filenames.shape[0])
    return filenames

def get_interesting_values(folder_path,filenames,interesting_fields,interesting_columns,field_column_name):
    dfs = []
    for fname in filenames:
        new_df = pd.read_csv(folder_path + fname,usecols=interesting_columns)
        for field in interesting_fields:
            dfs.append( new_df.loc[new_df[field_column_name] == field])
    return pd.concat(dfs)
        

# Guinea

In [300]:
data_folder = DATA_DIRECTORY + 'guinea_data/'
filenames_guinea = get_csv_in_chrono_order(data_folder)

#print((pd.read_csv(data_folder + filenames_guinea[1],usecols=['Date','Description','Totals'])))
#filenames_guinea.count


In [301]:
fields = ['New cases of suspects','New cases of probables','New cases of confirmed',
          'New deaths registered','Total deaths of suspects',
         'Total deaths of probables','Total deaths of confirmed']
new_cases = get_interesting_values(data_folder,filenames_guinea,fields,['Date','Description','Totals'],'Description')
new_cases = new_cases.set_index('Date')
colsname = new_cases['Description'].unique()

series = []
for elem in colsname:
    series.append(new_cases.loc[(new_cases['Description'] == elem)])
    series[len(series)-1] = series[len(series)-1].drop(['Description'],axis=1)
    series[len(series)-1] = series[len(series)-1].rename(columns={'Totals':elem})
    
    
guinea = pd.concat(series,axis=1,join='outer')

In [302]:
ordered_dates = [x[0:10] for x in filenames_guinea]
ordered_dates = pd.to_datetime(ordered_dates,format='%Y-%m-%d')
time_span = ordered_dates[ordered_dates.shape[0]-1] - ordered_dates[0]

# Liberia

In [303]:
data_folder = DATA_DIRECTORY + 'liberia_data/'
filenames_liberia = get_csv_in_chrono_order(data_folder)

#print(pd.read_csv(data_folder + filenames_liberia[21],usecols=['Date','Variable','National']))
#filenames_liberia.count

In [304]:
fields = ['New Case/s (Suspected)','New Case/s (Probable)','New case/s (confirmed)',
         'Total death/s in confirmed cases', 'Total death/s in probable cases',
          'Total death/s in suspected cases', 'Newly reported deaths']
new_cases = get_interesting_values(data_folder,filenames_liberia,fields,['Date','Variable','National'],'Variable')
new_cases = new_cases.set_index('Date')
colsname = new_cases['Variable'].unique()

series = []
for elem in colsname:
    series.append(new_cases.loc[(new_cases['Variable'] == elem)])
    series[len(series)-1] = series[len(series)-1].drop(['Variable'],axis=1)
    series[len(series)-1] = series[len(series)-1].rename(columns={'National':elem})
    
series   
liberia = pd.concat(series,axis=1,join='inner')

# Sierra Léon

In [305]:
data_folder = DATA_DIRECTORY + 'sl_data/'
filenames_sl = get_csv_in_chrono_order(data_folder)

#print(pd.read_csv(data_folder + filenames_sl[21],usecols=['date','variable','National']))
#filenames_sl.count

In [306]:
fields = ['new_suspected','new_probable','new_confirmed',
         'death_suspected', 'death_probable',
          'death_confirmed', 'etc_new_deaths']
new_cases = get_interesting_values(data_folder,filenames_sl,fields,['date','variable','National'],'variable')
new_cases = new_cases.set_index('date')
colsname = new_cases['variable'].unique()

series = []
for elem in colsname:
    series.append(new_cases.loc[(new_cases['variable'] == elem)])
    series[len(series)-1] = series[len(series)-1].drop(['variable'],axis=1)
    series[len(series)-1] = series[len(series)-1].rename(columns={'National':elem})
    
series   
sl = pd.concat(series,axis=1,join='inner')

###### Uniformisation of column names

In [307]:
guinea.head()

,New cases of suspects,New cases of probables,New cases of confirmed,Total deaths of suspects,Total deaths of probables,Total deaths of confirmed,New deaths registered
2014-08-04,5,0,4,2,133,228,NaN
2014-08-26,18,NaN,10,2,141,292,5
2014-08-27,12,0,10,2,142,294,2
2014-08-30,15,0,9,2,142,324,5
2014-08-31,9,8,29,2,150,337,3


In [308]:
#guinea['Country'] = ['Guinea']*guinea.shape[0]

guinea.columns
liberia.columns = guinea.columns
sl.columns = guinea.columns

###### Converting dates to datetime

In [309]:
#guinea.index = pd.to_datetime(guinea.index,format='%Y-%m-%d')
#liberia.index = pd.to_datetime(liberia.index,format='%m/%d/%Y')
#sl.index = pd.to_datetime(sl.index,format='%Y-%m-%d')
#sl.index[x for x in sl.shape[0] if 2014 not in sl.index[x]] = 


guinea.head()

,New cases of suspects,New cases of probables,New cases of confirmed,Total deaths of suspects,Total deaths of probables,Total deaths of confirmed,New deaths registered
2014-08-04,5,0,4,2,133,228,NaN
2014-08-26,18,NaN,10,2,141,292,5
2014-08-27,12,0,10,2,142,294,2
2014-08-30,15,0,9,2,142,324,5
2014-08-31,9,8,29,2,150,337,3


###### Add country column for each of the three DataFrame

In [310]:
guinea['Country'] = ['Guinea']*guinea.shape[0]
liberia['Country'] = ['Liberia']*liberia.shape[0]
sl['Country'] = ['Sierra Leone']*sl.shape[0]

###### Transferring date into column

In [311]:
guinea['Date'] = guinea.index
liberia['Date'] = liberia.index
sl['Date'] = sl.index

###### Summary

In [313]:
guinea.head()

,New cases of suspects,New cases of probables,New cases of confirmed,Total deaths of suspects,Total deaths of probables,Total deaths of confirmed,New deaths registered,Country,Date
2014-08-04,5,0,4,2,133,228,NaN,Guinea,2014-08-04
2014-08-26,18,NaN,10,2,141,292,5,Guinea,2014-08-26
2014-08-27,12,0,10,2,142,294,2,Guinea,2014-08-27
2014-08-30,15,0,9,2,142,324,5,Guinea,2014-08-30
2014-08-31,9,8,29,2,150,337,3,Guinea,2014-08-31


In [314]:
liberia.head()

,New cases of suspects,New cases of probables,New cases of confirmed,Total deaths of suspects,Total deaths of probables,Total deaths of confirmed,New deaths registered,Country,Date
Date,,,,,,,,,
6/16/2014,2.0,1.0,1.0,8.0,6.0,2.0,2.0,Liberia,6/16/2014
6/17/2014,2.0,0.0,0.0,8.0,6.0,2.0,0.0,Liberia,6/17/2014
6/22/2014,5.0,0.0,5.0,16.0,8.0,1.0,4.0,Liberia,6/22/2014
6/24/2014,2.0,0.0,4.0,18.0,8.0,6.0,4.0,Liberia,6/24/2014
6/25/14,4.0,1.0,2.0,20.0,8.0,9.0,3.0,Liberia,6/25/14


In [315]:
sl.head()

,New cases of suspects,New cases of probables,New cases of confirmed,Total deaths of suspects,Total deaths of probables,Total deaths of confirmed,New deaths registered,Country,Date
date,,,,,,,,,
2014-08-12,10,1,11,5,34,264,NaN,Sierra Leone,2014-08-12
2014-08-13,3,1,15,5,34,273,NaN,Sierra Leone,2014-08-13
2014-08-14,0,2,13,5,34,280,NaN,Sierra Leone,2014-08-14
2014-08-15,6,1,10,5,34,287,NaN,Sierra Leone,2014-08-15
2014-08-16,3,0,18,5,34,297,NaN,Sierra Leone,2014-08-16


###### Concatenation of the 3 DataFrames

In [316]:
ebola = pd.concat([guinea, liberia, sl], axis = 0)
ebola.head()

,New cases of suspects,New cases of probables,New cases of confirmed,Total deaths of suspects,Total deaths of probables,Total deaths of confirmed,New deaths registered,Country,Date
2014-08-04,5,0,4,2,133,228,NaN,Guinea,2014-08-04
2014-08-26,18,NaN,10,2,141,292,5,Guinea,2014-08-26
2014-08-27,12,0,10,2,142,294,2,Guinea,2014-08-27
2014-08-30,15,0,9,2,142,324,5,Guinea,2014-08-30
2014-08-31,9,8,29,2,150,337,3,Guinea,2014-08-31


In [317]:
ebola = ebola.set_index(['Country', 'Date'])
ebola.head()

New cases of suspects New cases of probables  \
Country Date                                                      
Guinea  2014-08-04                     5                      0   
        2014-08-26                    18                    NaN   
        2014-08-27                    12                      0   
        2014-08-30                    15                      0   
        2014-08-31                     9                      8   

                   New cases of confirmed Total deaths of suspects  \
Country Date                                                         
Guinea  2014-08-04                      4                        2   
        2014-08-26                     10                        2   
        2014-08-27                     10                        2   
        2014-08-30                      9                        2   
        2014-08-31                     29                        2   

                   Total deaths of probables Total deaths of confirmed  \
Country Date                                                             
Guinea  2014-08-04                       133                       228   
        2014-08-26                       141                       292   
        2014-08-27                       142                       294   
        2014-08-30                       142                       324   
        2014-08-31                       150                       337   

                   New deaths registered  
Country Date                              
Guinea  2014-08-04                   NaN  
        2014-08-26                     5  
        2014-08-27                     2  
        2014-08-30                     5  
        2014-08-31                     3

In [318]:
ebola.columns

Index(['New cases of suspects', 'New cases of probables',
       'New cases of confirmed', 'Total deaths of suspects',
       'Total deaths of probables', 'Total deaths of confirmed',
       'New deaths registered'],
      dtype='object')

In [331]:
new = ['suspects', 'probables', 'confirmed', 'suspects', 'probables', 'confirmed', 'registered']

In [332]:
ebola.columns = pd.MultiIndex.from_arrays([['New cases']*3 + ['Deaths']*4, new])
#ebola.columns = [['New cases', 'Suspects'], ['New cases', 'Probables'], ['New cases', 'Confirmed'],\
#                 'Total deaths of suspects', 'Total deaths of probables', 'Total deaths of confirmed', \
#                 'New deaths registered', 'Country', 'Date']


In [333]:
ebola.head()

New cases                       Deaths                      \
                    suspects probables confirmed suspects probables confirmed   
Country Date                                                                    
Guinea  2014-08-04         5         0         4        2       133       228   
        2014-08-26        18       NaN        10        2       141       292   
        2014-08-27        12         0        10        2       142       294   
        2014-08-30        15         0         9        2       142       324   
        2014-08-31         9         8        29        2       150       337   

                               
                   registered  
Country Date                   
Guinea  2014-08-04        NaN  
        2014-08-26          5  
        2014-08-27          2  
        2014-08-30          5  
        2014-08-31          3

:-)

In [384]:
serie_total_new_cases = ebola[('New cases', 'suspects')].astype(float) + ebola[('New cases', 'probables')].astype(float) + \
                ebola[('New cases', 'confirmed')].astype(float)

serie_daily_deaths = ebola[('Deaths', 'suspects')].astype(float) + ebola[('Deaths', 'probables')].astype(float) + \
                ebola[('Deaths', 'confirmed')].astype(float)

print('Average of total daily new cases for Guinea:', serie_total_new_cases['Guinea'].mean())
print('Average of total daily new cases for Liberia:', serie_total_new_cases['Liberia'].mean())
print('Average of total daily new cases for Sierra Leone:', serie_total_new_cases['Sierra Leone'].mean())
print()
print('Average of suspected daily new cases for Guinea:', ebola.loc[['Guinea']][('New cases', 'suspects')].astype(float).mean())
print('Average of suspected daily new cases for Liberia:', ebola.loc[['Liberia']][('New cases', 'suspects')].astype(float).mean())
print('Average of suspected daily new cases for Sierra Leone:', ebola.loc[['Sierra Leone']][('New cases', 'suspects')].astype(float).mean())
print()
print('Average of probables daily new cases for Guinea:', ebola.loc[['Guinea']][('New cases', 'probables')].astype(float).mean())
print('Average of probables daily new cases for Liberia:', ebola.loc[['Liberia']][('New cases', 'probables')].astype(float).mean())
print('Average of probables daily new cases for Sierra Leone:', ebola.loc[['Sierra Leone']][('New cases', 'probables')].astype(float).mean())
print()
print('Average of confirmed daily new cases for Guinea:', ebola.loc[['Guinea']][('New cases', 'confirmed')].astype(float).mean())
print('Average of confirmed daily new cases for Liberia:', ebola.loc[['Liberia']][('New cases', 'confirmed')].astype(float).mean())
print('Average of confirmed daily new cases for Sierra Leone:', ebola.loc[['Sierra Leone']][('New cases', 'confirmed')].astype(float).mean())
print()



print('Average of daily deaths for Guinea:', ebola.loc[['Guinea']][('Deaths','registered')].astype(float).mean())
print('Average of daily deaths for Liberia:', ebola.loc[['Liberia']][('Deaths','registered')].astype(float).mean())
print('Average of daily deaths for Sierra Leone:', ebola.loc[['Sierra Leone']][('Deaths','registered')].astype(float).mean())

print()






Average of total daily new cases for Guinea: 21.38095238095238
Average of total daily new cases for Liberia: 591.3809523809524
Average of total daily new cases for Sierra Leone: 57.57954545454545

Average of suspected daily new cases for Guinea: 7.909090909090909
Average of suspected daily new cases for Liberia: 205.66326530612244
Average of suspected daily new cases for Sierra Leone: 11.693181818181818

Average of probables daily new cases for Guinea: 1.2857142857142858
Average of probables daily new cases for Liberia: 126.18367346938776
Average of probables daily new cases for Sierra Leone: 0.5340909090909091

Average of confirmed daily new cases for Guinea: 12.545454545454545
Average of confirmed daily new cases for Liberia: 205.40697674418604
Average of confirmed daily new cases for Sierra Leone: 46.084210526315786

Average of daily deaths for Guinea: 4.142857142857143



## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here